In [1]:
#Verify to have all installed
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))

Python: 3.6.8 (default, Oct  7 2019, 12:59:55) 
[GCC 8.3.0]
scipy: 1.3.1
numpy: 1.17.2
matplotlib: 3.1.1
pandas: 0.25.1
sklearn: 0.21.3


In [2]:
# Load libraries
import pandas as pd
import datetime
from datetime import date,time
import time
import matplotlib.pyplot as plt

#Modelos simples (no uso todo)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
#from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression

#Modelos de prueba real
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge

#Otros modelos
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm
from sklearn.linear_model import SGDRegressor
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [3]:
#Carga y filtrado inicial de los datos
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
drop_cols = ['titulo', 'descripcion', 'direccion', 'idzona', 'lat', 'lng'] #-ciudad
train = train.drop(drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)

#drop del algoritmo
test = test.drop(['fecha'], axis=1)
x_train = train.drop(['precio', 'fecha'], axis=1)

numeric_columns_with_nulls = list(set(x_train.columns[x_train.isnull().sum() > 0].tolist()) \
                                  - set(['tiposdepropiedad', 'ciudad', 'provincia']))
numeric_columns_with_nulls = ['habitaciones',
 'garages',
 'banos',
 'antiguedad',
 'metrostotales',
 'metroscubiertos']


In [4]:
for c in numeric_columns_with_nulls:
    imp = SimpleImputer()
    x_train[c] = imp.fit_transform(x_train[[c]])
    test[c] = imp.transform(test[[c]])

In [5]:
#columns_with_values = x_train.describe().columns.delete(0)
#for c in list(set(columns_with_values)):
#    imp = StandardScaler()
#    x_train[c] = imp.fit_transform(x_train[[c]])
#    test[c] = imp.transform(test[[c]])

In [6]:
#Definición de tipos
x_train['id'] = train['id'].astype(int)
x_train['tipodepropiedad'] = train['tipodepropiedad'].astype('object')
x_train['ciudad'] = train['ciudad'].astype('object')
x_train['provincia'] = train['provincia'].astype('object')
x_train['habitaciones'] = train['habitaciones'].fillna(0.0).astype(int)
x_train['garages'] = train['garages'].fillna(0.0).astype(int)
x_train['banos'] = train['banos'].fillna(0.0).astype(int)
#x_train['fecha'] = pd.to_datetime(train['fecha'], errors='coerce')
x_train['gimnasio'] = train['gimnasio'].astype('bool')
x_train['usosmultiples'] = train['usosmultiples'].astype('bool')
x_train['piscina'] = train['piscina'].fillna(0.0).astype('bool')
x_train['escuelascercanas'] = train['escuelascercanas'].astype('bool')
x_train['centroscomercialescercanos'] = train['centroscomercialescercanos'].astype('bool')

test['id'] = test['id'].astype(int)
test['tipodepropiedad'] = test['tipodepropiedad'].astype('object')
test['ciudad'] = test['ciudad'].astype('object')
test['provincia'] = test['provincia'].astype('object')
test['habitaciones'] = test['habitaciones'].fillna(0.0).astype(int)
test['garages'] = test['garages'].fillna(0.0).astype(int)
test['banos'] = test['banos'].fillna(0.0).astype(int)
#test['fecha'] = pd.to_datetime(test['fecha'], errors='coerce')
test['gimnasio'] = test['gimnasio'].fillna(0.0).astype('bool')
test['usosmultiples'] = test['usosmultiples'].fillna(0.0).astype('bool')
test['piscina'] = test['piscina'].fillna(0.0).astype('bool')
test['escuelascercanas'] = test['escuelascercanas'].fillna(0.0).astype('bool')
test['centroscomercialescercanos'] = test['centroscomercialescercanos'].fillna(0.0).astype('bool')

In [7]:
#Analisis preeliminar sobre test
#print('Se ofrecen ',len(train['tipodepropiedad'].unique()), 'tipos de propíedades:')
#((test['banos'].value_counts()/60000)-(train['banos'].value_counts()/240000))*100
#anuncios_fecha = test['antiguedad']
#anuncios_fechaOT = train['antiguedad']
#((anuncios_fecha.mean()/anuncios_fechaOT.mean())-1)*100
#(((anuncios_fecha.value_counts()/60000)-(anuncios_fechaOT.value_counts()/240000))*100).apply(lambda x: x if x > 0 else -x).sum()

In [8]:
#Analisis de test data
#((60000-test.isnull().sum())/60000)*100
#print(test['tipodepropiedad'].unique())
#print(f"Total tipos de propiedad: {test['id'].nunique()}")
#print(f"Total tipos de propiedad: {test['tipodepropiedad'].nunique()}")
#print(f"Total tipos de propiedad: {test['ciudad'].nunique()}")
#print(f"Total tipos de propiedad: {test['provincia'].nunique()}")
#print(f"Total tipos de propiedad: {test['antiguedad'].nunique()}")
#print(f"Total tipos de propiedad: {test['habitaciones'].nunique()}")
#print(f"Total tipos de propiedad: {test['garages'].nunique()}")
#print(f"Total tipos de propiedad: {test['banos'].nunique()}")
#print(f"Total tipos de propiedad: {test['metroscubiertos'].nunique()}")
#print(f"Total tipos de propiedad: {test['metroscubiertos'].nunique()}")
#print(f"Total tipos de gilads: {test['fecha'].nunique()}")
#print(f"Total tipos de propiedad: {test['usosmultiples'].nunique()}")
#print(f"Total tipos de propiedad: {test['piscina'].nunique()}")
#print(f"Total tipos de propiedad: {test['escuelascercanas'].nunique()}")
#print(f"Total tipos de propiedad: {test['centroscomercialescercanos'].nunique()}")

In [9]:
#Dummy Regressor

# Tiramos todas las columnas no numéricas para que sklearn nos acepte el dataframe. 
# Igualmente, el DummyRegressor no va a utilizar ninguna más que el precio 
#drop_cols = ['fecha', 'ciudad', 'idzona', 'tipodepropiedad', 'provincia', 'titulo', 'descripcion', 'direccion']
#X = train.drop(['precio'] + drop_cols, axis=1)
#y = train['precio']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
#print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
#print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")
#dummy = DummyRegressor(strategy='mean').fit(X_train, y_train)
#pred = dummy.predict(X_test)
#predictions = pd.DataFrame(pred)
#predictions.columns = ['target']
#predictions['id'] = test['id']
#predictions.to_csv('dummy.csv',index=False)

In [10]:
### Regresion lineal sobre metros totales
#x_train = pd.DataFrame(train, columns=['id', 'metrostotales'])
#x_train.set_index('id')
#y_train = pd.DataFrame(train, columns=['id', 'precio'])
#y_train.set_index('id')
#x_test = pd.DataFrame(test, columns=['id', 'metrostotales'])
#x_test.set_index('id')
#Reemplazo null por el promedio
#imp = SimpleImputer()
#x_train['metrostotales'] = imp.fit_transform(x_train[['metrostotales']])
#x_test['metrostotales'] = imp.transform(x_test[['metrostotales']])
#x_test.groupby('id')
#linear_model = LogisticRegression().fit(x_train, y_train['precio'])
#predicciones = pd.DataFrame()
#data = linear_model.predict(x_test)
#predicciones['id'] = pd.to_numeric(data[0:,0]).round(0)
#predicciones['target'] = data[0:,1]
#predicciones['id'] = predicciones['id'].astype(int)
#predicciones.to_csv('LogisticRegressionOnMtsOutside.csv',index=False)

In [11]:
#Algoritmos
tree_model = DecisionTreeRegressor()
rf_model = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
#lasso = Lasso(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=3000,
      #normalize=False, positive=True, precompute=False, random_state=None)
#elastic_net = ElasticNet()
#ridge = Ridge(alpha=10, copy_X=True, fit_intercept=False, max_iter=None)
#svr = svm.SVR(kernel='linear',gamma='auto')
sgd = linear_model.SGDRegressor(max_iter=3000, loss='epsilon_insensitive')

In [12]:
#Transformacion de los datos One-hot encoding
y_train = train['precio']
x_train = pd.get_dummies(x_train, columns=['tipodepropiedad', 'provincia', 'ciudad'])

In [13]:
test = pd.get_dummies(test, columns=['tipodepropiedad','provincia', 'ciudad'])

In [14]:
a = test.columns.intersection(x_train.columns)
test = test[a]

In [15]:
x_train = x_train[a]

In [16]:
#Aplico Random Forest
#rf_model.fit(x_train, train['precio'])

In [17]:
#test.columns.unique()
#tree_pred = rf_model.predict(test)
#predicciones = pd.DataFrame()

#predicciones['target'] = tree_pred
#predicciones['id']=test['id']
#predicciones.to_csv('RandomForestWithOneHotTodos300Regressor.csv', index=False)

In [18]:
# Algoritmo Decision Tree

#tree = DecisionTreeRegressor().fit(x_train, y_train)
#tree_pred = tree.predict(test)
#predicciones = pd.DataFrame()
#predicciones['target'] = tree_pred
#predicciones['id']=test['id']
#predicciones.to_csv('DecisionTreeRegressor.csv',index=False)

In [19]:
# Algoritmo Lasso o Elastic Net
#lasso.fit(x_train, y_train)
#elastic_net.fit(x_train, y_train)

In [20]:
#tree_pred = lasso.predict(test)
#tree_pred = elastic_net.predict(test)
#predicciones = pd.DataFrame()
#tree_pred
#predicciones['target'] = tree_pred
#predicciones['id']=test['id']
#predicciones.to_csv('Lasso3000Iter.csv', index=False)

In [21]:
#Algoritmo Ridge regressor con testeo
#ridge.fit(x_train, train['precio'])
#ridge_pred = ridge.predict(test)
#predicciones = pd.DataFrame()
#predicciones['target'] = ridge_pred
#predicciones['id']=test['id']
#predicciones.to_csv('RidgeRegressor.csv',index=False)

In [22]:
#poly = PolynomialFeatures(degree=3)
#X_poly = poly.fit_transform(x_train)
#X_train, X_test, y_train, y_test = train_test_split(
#    x_train, train['precio'], test_size=0.3, random_state=0)

#for a in [0, 1, 10, 100, 1000]: #, 10, 100, 1000]:
#    lrRg = Ridge(alpha=a)
#    lrRg.fit(X_train, y_train)

#    y_train_pred = lrRg.predict(X_train)
#    y_test_pred = lrRg.predict(X_test)

#    print('\n[Alpha = %d]' % a )
#    print('MSE train: %.2f, test: %.2f' % (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)))

In [23]:
#Algoritmo SVR (dado de baja por tardar mas de 10hs sin concluir)
#svr.fit(x_train, train['precio'])
#svr.predict(test)

In [24]:
#Algoritmo SGDRegressor
#sgd.fit(x_train, train['precio'])
#sgd_pred = sgd.predict(test)
#sgd_pred

In [25]:
#predicciones = pd.DataFrame()
#predicciones['target'] = sgd_pred
#predicciones['id']=test['id']
#predicciones.to_csv('SGDRegressorLastTry.csv',index=False)

In [26]:
#train['precio']

In [27]:
#X_train, X_test, y_train, y_test = train_test_split(
 #   x_train, train['precio'], test_size=0.3, random_state=0)

#for a in [1000, 10000, 100000]: #, 10, 100, 1000]:
 #   sgdReg = linear_model.SGDRegressor(alpha = a, max_iter=1000, tol=1e-3, loss='epsilon_insensitive')
  #  sgdReg.fit(X_train, y_train)

  #  y_train_pred = sgdReg.predict(X_train)
   # y_test_pred = sgdReg.predict(X_test)

    #print('\n[Alpha = %d]' % a )
    #print('MSE train: %.2f, test: %.2f' % (mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)))

In [28]:
#Algoritmo XGBBoost

#for f in x_train.columns:
 #   if x_train[f].dtype == 'object':
  #      label = LabelEncoder()
   #     label.fit(list(x_train[f].values))
    #    x_train[f] = label.transform(list(x_train[f].values))
        #test[f] = label.transform(list(x_train[f].values))

In [29]:
#for f in test.columns:
 #   if test[f].dtype == 'object':
  #      label = LabelEncoder()
   #     label.fit(list(test[f].values))
    #    test[f] = label.transform(list(test[f].values))

In [30]:
#model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#             colsample_bynode=1, colsample_bytree=1, gamma=0,
#             importance_type='gain', learning_rate=0.1, max_delta_step=0,
#             max_depth=2,min_child_weight=1, missing=None, n_estimators=10000,
#             n_jobs=-1, nthread=None, random_state=0,
#             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#             silent=None, subsample=1, verbosity=1)
#model.fit(x_train,train['precio'])

In [31]:
#xgb_pred = model.predict(test)
#predicciones = pd.DataFrame()
#xgb_pred
#predicciones['target'] = xgb_pred
#predicciones['id'] = test['id']
#predicciones.to_csv('XGBBoost.csv', index=False)

In [32]:
#Aplico Random Forest
rf_model.fit(x_train, train['precio'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [35]:
tree_pred = rf_model.predict(test)
predicciones = pd.DataFrame()

predicciones['target'] = tree_pred
predicciones['id']=test['id']
predicciones.to_csv('RandomForest300Regressor.csv', index=False)

In [34]:
#Análisis de resultados
#alg = ['LinearMtsInside',
# 'LinearMtsOutside',
# 'DecissionTree',
# 'RandomForest100Trees',
# 'RandomForest300Trees',
# 'RandomForestProvinceHotEncoding100Trees',
# 'RandomForestAllHotEncoding100Trees',
# 'RandomForestAllHotEncoding200Trees',
# 'Lasso3kIters',
# 'ElasticNet1kIters',
# 'Ridge',
# 'SGD',
# 'RandomForestOtherParams',
# 'SGDBetter',
# 'RandomForestAllHotEncoding300Trees',
# 'XGB Boost',
# 'RandomForestAllLabelEncoding300Trees']
#value = [1191313,
# 1372762,
# 1275505,
# 954552,
# 951180,
# 891231,
# 657990,
# 657451,
# 956340,
# 1033779,
# 1076081,
# 1915092,
 #1252704,
 #1100974,
 #656154,
 #898952,
 #830720]
#plt.plot(alg, value)
#plt.xticks(rotation='vertical')